In [23]:
import sys, os
sys.path.append(os.path.abspath(".."))

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

In [1]:
import pandas as pd

In [2]:
df_long = pd.read_csv("../data/processed/crimes_against_women_long.csv")

## Aggregate per year per crime type

In [3]:
yearly_crime = (
    df_long
    .groupby(['Year','Crime_Type'])['Crime_Count']
    .sum()
    .reset_index()
)

In [4]:
rape_ts = yearly_crime[yearly_crime['Crime_Type'] == 'Rape']
rape_ts = rape_ts.sort_values('Year')

rape_ts


Year Crime_Type  Crime_Count
6   2001       Rape        32150
13  2002       Rape        32746
20  2003       Rape        31694
27  2004       Rape        36466
34  2005       Rape        36718
41  2006       Rape        38696
48  2007       Rape        41474
55  2008       Rape        42934
62  2009       Rape        42794
69  2010       Rape        44344
76  2011       Rape        48412
83  2012       Rape        49846
90  2013       Rape        67414
97  2014       Rape        73470

## Normalize values (LSTM requirement)

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
values = rape_ts['Crime_Count'].values.reshape(-1,1)
values_scaled = scaler.fit_transform(values)

## Create sequences

In [8]:
def create_sequences(data,window=3):
    X,y = [],[],
    for i in range (len(data) - window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

import numpy as np

X, y = create_sequences(values_scaled,window = 3)

## Build LSTM

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(32, activation='tanh', input_shape=(X.shape[1],1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

E:\Visual Studio code\BIA\Gen ai\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Train

In [12]:
model.fit(X, y, epochs=200, verbose=0)

## Predict future (next 3 years)

In [14]:
last_window = values_scaled[-3:]
predictions = []

current = last_window.copy()

for _ in range(3):
    pred = model.predict(current.reshape(1, 3, 1), verbose=0)
    predictions.append(pred[0,0])
    current = np.vstack([current[1:], pred])

predictions = scaler.inverse_transform(
    np.array(predictions).reshape(-1,1)
)

predictions


array([[ 91652.03],
       [132095.61],
       [171943.11]], dtype=float32)

In [15]:
X.shape

(11, 3, 1)

In [16]:
X = X.reshape((X.shape[0], X.shape[1], 1))


In [17]:
predictions


array([[ 91652.03],
       [132095.61],
       [171943.11]], dtype=float32)

## Testing

In [24]:
district_clusters = pd.read_csv(
    "../data/processed/district_crime_clusters.csv"
)

In [26]:
cluster_summary = (
    district_clusters
    .groupby("Cluster")
    .mean(numeric_only=True)
)


In [27]:
cluster_summary.head()

Assault on women with intent to outrage her modesty  \
Cluster                                                        
0                                               298.948697     
1                                             44964.000000     
2                                              3895.333333     
3                                             29698.000000     
4                                             14298.357143     

         Cruelty by Husband or his Relatives  Dowry Deaths  \
Cluster                                                      
0                                 540.377555     47.952305   
1                               94021.600000   5947.600000   
2                               11564.666667   5778.666667   
3                               77617.000000  23824.000000   
4                               21659.071429   1781.500000   

         Importation of Girls  Insult to modesty of Women  \
Cluster                                                     
0                    0.311824                   68.473747   
1                   39.800000                11893.200000   
2                  265.333333                  168.333333   
3                    3.000000                26511.000000   
4                    6.857143                 2530.857143   

         Kidnapping and Abduction          Rape  
Cluster                                          
0                      185.805611    151.236072  
1                    16295.200000  20827.400000  
2                     8368.000000   7555.000000  
3                    47590.000000  19058.000000  
4                     9174.500000   6854.571429

In [28]:
cluster_summary.shape

(5, 7)

In [29]:
from src.genai_explainer import generate_crime_insight

insight = generate_crime_insight(
    crime_type="Rape",
    trend_values=[91652, 132096, 171943],
    cluster_summary=cluster_summary
)

print(insight)


Crime Analysis Summary for Rape

• The overall trend is increasing over the forecasted period.
• The highest concentration of related crimes appears in Cluster 3.
• This suggests systemic and persistent factors rather than short-term fluctuations.
• Policy focus should prioritize districts within this cluster for targeted intervention.
